<a href="https://colab.research.google.com/github/surjit84/ApacheBeamHandsOn/blob/main/Creating_Composite_Transform%2C_CoGroupByKey%2CsideInput%2CAdditionalOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 28.3 MB

In [2]:
import apache_beam as beam

class MyTransform(beam.PTransform):
  def expand(self, input_coll):
    a = (
        input_coll
        |"Map account key with value 1" >> beam.Map(lambda record: (record[1], 1))
        |"Count sum of account key" >> beam.CombinePerKey(sum)
    )
    return a

p1 = beam.Pipeline()
input_collection =(
    p1
    |"Read from Text file" >> beam.io.ReadFromText('dept_data.txt')
    |"Split record with comma seperator" >> beam.Map(lambda record: record.split(','))
)
attendance_count =(
    input_collection
    |"Filter record for Account" >> beam.Filter(lambda record: record[3]=='Accounts')
    |"Composite Account Transform" >> MyTransform()
    |"Write output to account_output" >> beam.io.WriteToText('data/account_output')
)
hr_count =(
    input_collection
    |"Filter record for HR" >> beam.Filter(lambda record: record[3]=='HR')
    |"Composite HR Transform" >> MyTransform()
    |"Write output to hr_output" >> beam.io.WriteToText('data/hr_output')
)
p1.run()

In [3]:
print('Account count:')
!{('head -n 20 data/account_output-00000-of-00001')}
print('HR count:')
!{('head -n 20 data/hr_output-00000-of-00001')}

Account count:
('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)
HR count:
('Beryl', 62)
('Olga', 31)
('Leslie', 31)
('Mindy', 31)
('Vicky', 31)
('Richard', 31)
('Kirk', 31)
('Kaori', 31)
('Oscar', 31)


CoGroupByKey --> To Perform Join between two PCollection

In [8]:
import apache_beam as beam

def retTuple(element):
  thisTuple = element.split(',')
  return (thisTuple[0], thisTuple[1:])

p = beam.Pipeline()

dep_rows = (
    p
      |"Read Dept file" >> beam.io.ReadFromText('/content/drive/MyDrive/data/dept_data.txt')
      |"Pair each dep key" >> beam.Map(retTuple)
)
loc_rows = (
    p
      |"Read Loc file" >> beam.io.ReadFromText('/content/drive/MyDrive/data/location.txt')
      |"Pair each location key" >> beam.Map(retTuple)
)

result_rows =({"dep_rows":dep_rows, "loc_rows":loc_rows}
              |beam.CoGroupByKey()
              |beam.io.WriteToText('data/join_output')

)
p.run()

In [9]:
!{('head -n 20 data/join_output-00000-of-00001')}

('149633CM', {'dep_rows': [['Marco', '10', 'Accounts', '1-01-2019'], ['Marco', '10', 'Accounts', '2-01-2019'], ['Marco', '10', 'Accounts', '3-01-2019'], ['Marco', '10', 'Accounts', '4-01-2019'], ['Marco', '10', 'Accounts', '5-01-2019'], ['Marco', '10', 'Accounts', '6-01-2019'], ['Marco', '10', 'Accounts', '7-01-2019'], ['Marco', '10', 'Accounts', '8-01-2019'], ['Marco', '10', 'Accounts', '9-01-2019'], ['Marco', '10', 'Accounts', '10-01-2019'], ['Marco', '10', 'Accounts', '11-01-2019'], ['Marco', '10', 'Accounts', '12-01-2019'], ['Marco', '10', 'Accounts', '13-01-2019'], ['Marco', '10', 'Accounts', '14-01-2019'], ['Marco', '10', 'Accounts', '15-01-2019'], ['Marco', '10', 'Accounts', '16-01-2019'], ['Marco', '10', 'Accounts', '17-01-2019'], ['Marco', '10', 'Accounts', '18-01-2019'], ['Marco', '10', 'Accounts', '19-01-2019'], ['Marco', '10', 'Accounts', '20-01-2019'], ['Marco', '10', 'Accounts', '21-01-2019'], ['Marco', '10', 'Accounts', '22-01-2019'], ['Marco', '10', 'Accounts', '23-01-2

In [7]:
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [11]:
import apache_beam as beam

side_list=list()
with open ('/content/drive/MyDrive/data/exclude_ids.txt','r') as my_file:
  for line in my_file:
    side_list.append(line.rstrip())

p = beam.Pipeline()

# We can pass side inputs to a ParDo transform, which will get passed to its process method.
# The first two arguments for the process method would be self and element.

class FilterUsingLength(beam.DoFn):
  def process(self, element,side_list,lower_bound, upper_bound=float('inf')):
    id = element.split(',')[0]
    name = element.split(',')[1]
    #id=id.decode('utf-8','ignore').encode("utf-8")
    element_list= element.split(',')
    if (lower_bound <= len(name) <= upper_bound) and id not in side_list:
      return [element_list]

# using pardo to filter names with length between 3 and 10
small_names =(
                p
                | "Read from text file" >> beam.io.ReadFromText('/content/drive/MyDrive/data/dept_data.txt')
                | "ParDo with side inputs" >> beam.ParDo(FilterUsingLength(),side_list,3,10)
                | beam.Filter(lambda record: record[3] == 'Accounts')
                | beam.Map(lambda record: (record[0]+ " " + record[1], 1))
                | beam.CombinePerKey(sum)
                | 'Write results' >> beam.io.WriteToText('data/output_new_final')
             )

p.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)


Additional Output

In [13]:
import apache_beam as beam

class ProcessWords(beam.DoFn):
  def process(self, element, cutoff_length, marker):
    name = element.split(',')[1]
    if len(name) <= cutoff_length:
      return [beam.pvalue.TaggedOutput('Short_Names',name)]
    else:
      return [beam.pvalue.TaggedOutput('Long_Names',name)]
    if name.startswith(marker):
      return name

p = beam.Pipeline()

results = (
    p
      |beam.io.ReadFromText('/content/drive/MyDrive/data/dept_data.txt')
      |beam.ParDo(ProcessWords(),cutoff_length=4,marker='A').with_outputs('Short_Names','Long_Names', main='Names_A')
)

short_collection = results.Short_Names
long_collection = results.Long_Names
startA_collection = results.Names_A

#Write to File
short_collection | 'Write 1' >> beam.io.WriteToText('data/short')
long_collection | 'Write 2' >> beam.io.WriteToText('data/long')
startA_collection | 'Write 3' >> beam.io.WriteToText('data/startA')

p.run()

print('short:\n')
!{('head -n 20 data/short-00000-of-00001')}
print('long:\n')
!{('head -n 20 data/long-00000-of-00001')}
print('startA:\n')
!{('head -n 20 data/startA-00000-of-00001')}

short:

Itoe
Kyle
Kyle
Olga
Kirk
Itoe
Kyle
Kyle
Olga
Kirk
Itoe
Kyle
Kyle
Olga
Kirk
Itoe
Kyle
Kyle
Olga
Kirk
long:

Marco
Rebekah
Edouard
Kumiko
Gaston
Beryl
Leslie
Mindy
Vicky
Richard
Kaori
Beryl
Oscar
Kumiko
Wendy
Cristobal
Erika
Sebastien
Valerie
Dolly
startA:

